### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import itertools
# Used for mathematical functions
import math             
import logging

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
# import pyproj
# import shapely.geometry
# from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt
import tqdm


import plotly.express as px
from plotly.subplots import make_subplots

# OpenTNSim
import opentnsim

# turn on debug messages if want to see what's going on in detail
# logging.basicConfig(level=logging.DEBUG)
# turn off debug messages, only show info and higher level messages
logging.basicConfig(level=logging.INFO)

### Create vessel - add VesselProperties and ConsumesEnergy mixins

In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.energy.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [3]:
# Create a dict with all important settings

data_vessel = {
    "env": None,
    "name": 'Vessel M6',
    "route": None,
    "geometry": None,
    "v": None,  # m/s
    "V_g_ave":3,
    "V_g_profile":False,
    "type": None,
    "B": 9.5,
    "L": 85,
    "H_e": None, 
    "H_f": None, 
    "T": 2,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": False, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Dry_DH", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "P_installed": 780.0,   
    "P_tot_given": None, # kW
    "P_tot_given_profile":False,
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": False,
    "sailing_upstream":False,
    "wind_influence": False,
    "P_hotel_perc": 0,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 1,# number of propellers
    "L_w": 2.0 ,
    "C_B":0.85, 
    "C_year": 1961,
}             



In [4]:
data_vessel_i = data_vessel.copy()
vessel = TransportResource(**data_vessel_i)

### Set vessel properties and sailing conditions ( input value )

In [5]:
V_s = np.linspace(2.5, 5.5, 15)          # ship sailing speeds to water, (m/s)
h_0 = [3,3.3,4.2,5,8]                 # water depths,(m)
T = [2]   # draught


### prepare input matrix for calculation

In [6]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product(T, h_0, V_s))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"T": item[0], "h_0": item[1], "V_s": item[2]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.tail()

,T,h_0,V_s
70,2,8.0,4.642857
71,2,8.0,4.857143
72,2,8.0,5.071429
73,2,8.0,5.285714
74,2,8.0,5.500000


### Run OpenTNSim to Calculate resistance, required power and emissions 

In [7]:

results = []

for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    
    data_vessel_i = data_vessel.copy()
    
    vessel = TransportResource(**data_vessel_i)
    vessel._T = row['T']
    V_s = row['V_s']
    h_0 = row['h_0']
    vessel.calculate_properties() # L is used here in the computation of L_R
    # h_0 = vessel.calculate_h_squat(v = V_s, h_0 = h_0)
    # print(h_0)
    R_f = vessel.calculate_frictional_resistance(V_s, h_0) 
    R_f_one_k1 = vessel.calculate_viscous_resistance()
    R_APP = vessel.calculate_appendage_resistance(V_s)
    R_W = vessel.calculate_wave_resistance(V_s, h_0)
    R_res = vessel.calculate_residual_resistance(V_s, h_0)
    R_T = vessel.calculate_total_resistance(V_s, h_0)
    P_tot = vessel.calculate_total_power_required(V_s, h_0)

    
    result = {}
    result.update(row)
    result['P_installed'] = vessel.P_installed
    result['R_f'] = R_f
    result['R_f_one_k1'] = R_f_one_k1
    result['R_APP'] = R_APP
    result['R_W'] = R_W
    result['R_res'] = R_res
    result['R_T'] = R_T
    result['P_tot'] = P_tot

    results.append(result)

75it [00:00, 1110.76it/s]


In [8]:
plot_df = pd.DataFrame(results)


# convert from meters per second to km per hour
ms_to_kmh = 3.6
plot_df['V_s_km'] = plot_df['V_s'] * ms_to_kmh

plot_df.head(50)

,T,h_0,V_s,P_installed,R_f,R_f_one_k1,R_APP,R_W,R_res,R_T,P_tot,V_s_km
0,2.0,3.0,2.500000,780.0,6.497648,7.977764,0.812206,0.064543,3.759200,12.613713,"(114.67011638280508, 114.67011638280508, 114.6...",9.000000
1,2.0,3.0,2.714286,780.0,7.563447,9.286345,0.945431,0.148854,4.372217,14.752847,"(145.6125146965002, 145.6125146965002, 145.612...",9.771429
2,2.0,3.0,2.928571,780.0,8.703670,10.686303,1.087959,0.305482,5.021093,17.100836,"(182.11279592874808, 182.11279592874808, 182.1...",10.542857
3,2.0,3.0,3.142857,780.0,9.917581,12.176733,1.239698,0.570902,5.703619,19.690951,"(221.02087630238924, 221.02087630238924, 221.0...",11.314286
4,2.0,3.0,3.357143,780.0,11.204504,13.756807,1.400563,0.989787,6.417588,22.564745,"(270.5466886714585, 270.5466886714585, 270.546...",12.085714
5,2.0,3.0,3.571429,780.0,12.563819,15.425765,1.570477,1.606255,7.160793,25.763291,"(328.6134010378077, 328.6134010378077, 328.613...",12.857143
6,2.0,3.0,3.785714,780.0,13.994954,17.182901,1.749369,2.474785,7.931025,29.338081,"(396.6628254225182, 396.6628254225182, 396.662...",13.628571
7,2.0,3.0,4.000000,780.0,15.497373,19.027560,1.937172,3.641414,8.726077,33.332223,"(476.17461629879244, 476.17461629879244, 476.1...",14.400000
8,2.0,3.0,4.214286,780.0,17.070578,20.959130,2.133822,5.202161,9.543741,37.838855,"(569.5133782066326, 569.5133782066326, 569.513...",15.171429
9,2.0,3.0,4.428571,780.0,18.714102,22.977038,2.339263,7.315555,10.381810,43.013666,"(680.3181853136572, 680.3181853136572, 680.318...",15.942857


In [9]:
plot_df.tail(25)

,T,h_0,V_s,P_installed,R_f,R_f_one_k1,R_APP,R_W,R_res,R_T,P_tot,V_s_km
50,2.0,5.0,3.571429,780.0,11.559965,14.193240,1.444996,1.606255,7.160793,24.405283,"(164.45608652298685, 164.45608652298685, 164.4...",12.857143
51,2.0,5.0,3.785714,780.0,12.882542,15.817091,1.610318,2.474785,7.931025,27.833219,"(154.95384450441856, 154.95384450441856, 154.9...",13.628571
52,2.0,5.0,4.000000,780.0,14.271540,17.522492,1.783943,3.641414,8.726077,31.673926,"(180.99386314195672, 180.99386314195672, 180.9...",14.400000
53,2.0,5.0,4.214286,780.0,15.726513,19.308898,1.965814,5.202161,9.543741,36.020614,"(223.23700054967833, 223.23700054967833, 223.2...",15.171429
54,2.0,5.0,4.428571,780.0,17.247045,21.175795,2.155881,7.315555,10.381810,41.029041,"(259.57148558588204, 259.57148558588204, 259.5...",15.942857
55,2.0,5.0,4.642857,780.0,18.832742,23.122703,2.354093,9.360112,11.238076,46.074984,"(305.59938334772664, 305.59938334772664, 305.5...",16.714286
56,2.0,5.0,4.857143,780.0,20.483236,25.149167,2.560404,13.006671,12.110330,52.826573,"(366.551729617952, 366.551729617952, 366.55172...",17.485714
57,2.0,5.0,5.071429,780.0,22.198175,27.254756,2.774772,15.229607,12.996366,58.255502,"(422.0551690109704, 422.0551690109704, 422.055...",18.257143
58,2.0,5.0,5.285714,780.0,23.977228,29.439064,2.997153,21.327726,13.893976,67.657921,"(510.886338923067, 510.886338923067, 510.88633...",19.028571
59,2.0,5.0,5.500000,780.0,25.820079,31.701703,3.227510,24.639982,14.800952,74.370147,"(584.3368665579482, 584.3368665579482, 584.336...",19.800000
